In [10]:
class Matrix:
    def __init__(self, data):
        """
        初始化矩阵。
        :param data: 可选，用于初始化矩阵的数据列表
        """
        if not data or not data[0]:
            raise ValueError("数据为空")

        self.rows = len(data)
        self.cols = len(data[0])

        if any(len(r) != self.cols for r in data):
            raise ValueError("每一行长度不一致")

        self.data = data

    def __str__(self):
        """
        返回矩阵的字符串表示形式。
        """
        return '\n'.join([' '.join(map(str, row)) for row in self.data])

    def __sub__(self, other):
        """
        矩阵减法。

        :param other: 另一个矩阵。
        :return: 新的矩阵，表示两个矩阵的差。
        """
        if self.rows != other.rows or self.cols != other.cols:
            raise ValueError("矩阵尺寸不匹配，无法进行减法")
        result = Matrix([[0 for _ in range(self.cols)] for _ in range(self.rows)])
        for i in range(self.rows):
            for j in range(self.cols):
                result.data[i][j] = self.data[i][j] - other.data[i][j]
        return result


    def __add__(self, other):
        """
        实现矩阵加法。
        :param other: 另一个矩阵
        :return: 新矩阵，表示两个矩阵的和
        """
        if self.rows != other.rows or self.cols != other.cols:
            raise ValueError("矩阵尺寸不匹配，无法进行加法")
        return Matrix([[self.data[i][j] + other.data[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def mul(self, scalar):
        """
        矩阵乘以标量。

        :param scalar: 标量值。
        :return: 新的矩阵，表示矩阵与标量的乘积。
        """
        if not isinstance(scalar, (int, float)):
            raise ValueError("只能与标量值相乘")
        result = Matrix([[0 for _ in range(self.cols)] for _ in range(self.rows)])
        for i in range(self.rows):
            for j in range(self.cols):
                result.data[i][j] = self.data[i][j] * scalar
        return result

    def __mul__(self, scalar):
        return self.mul(scalar)

    def __rmul__(self, scalar):
        return self.mul(scalar)

    def __truediv__(self, scalar):
        """
        矩阵除以标量。

        :param scalar: 标量值。
        :return: 新的矩阵，表示矩阵与标量的商。
        """
        if not isinstance(scalar, (int, float)):
            raise ValueError("只能与标量值相除")
        if scalar == 0:
            raise ZeroDivisionError("不能除以零")
        result = Matrix([[0 for _ in range(self.cols)] for _ in range(self.rows)])
        for i in range(self.rows):
            for j in range(self.cols):
                result.data[i][j] = self.data[i][j] / scalar
        return result

    def __matmul__(self, other):
        """
        实现矩阵点积。
        :param other: 另一个矩阵
        :return: 新矩阵，表示两个矩阵的点积
        """
        if self.cols != other.rows:
            raise ValueError("矩阵尺寸不匹配，无法进行点积")
        result_data = [[0 for _ in range(other.cols)] for _ in range(self.rows)]
        for i in range(self.rows):
            for j in range(other.cols):
                for k in range(self.cols):
                    result_data[i][j] += self.data[i][k] * other.data[k][j]
        return Matrix(result_data)

    @property
    def T(self):
        """
        返回矩阵的转置。
        :return: 新矩阵，表示原矩阵的转置
        """
        return Matrix([[self.data[j][i] for j in range(self.rows)] for i in range(self.cols)])

    @property
    def Inv(self):
        if self.rows != self.cols:
            raise ValueError("只有方阵才有逆矩阵")

        n = self.rows

        # 创建增广矩阵（原矩阵与单位矩阵并排）
        augmented_matrix = [ [0 for __ in range(2 * n)] for _ in range(n)]
        for i in range(n):
            for j in range(n):
                augmented_matrix[i][j] = self.data[i][j]  # 填充原矩阵
                augmented_matrix[i][j + n] = 1 if i == j else 0  # 填充单位矩阵

        # 高斯-约尔当消元法
        for i in range(n):
            # 如果当前列的主元（对角线元素）为0，则通过行交换找到一个非0主元
            if augmented_matrix[i][i] == 0:
                for k in range(i + 1, n):
                    if augmented_matrix[k][i] != 0:
                        augmented_matrix[i], augmented_matrix[k] = augmented_matrix[k], augmented_matrix[i]
                        break
                else:
                    # 如果当前列的所有元素都为0，则矩阵不可逆
                    return None

            # 将当前行的主元归一化（即将主元变为1，并消去该行其他列的元素）
            pivot = augmented_matrix[i][i]
            for j in range(2 * n):
                augmented_matrix[i][j] /= pivot

            # 消去其他行的当前列元素
            for k in range(n):
                if k != i:
                    factor = augmented_matrix[k][i]
                    for j in range(2 * n):
                        augmented_matrix[k][j] -= factor * augmented_matrix[i][j]

        # 提取逆矩阵（增广矩阵的右半部分）
        inverse = [[augmented_matrix[i][j] for j in range(n, 2 * n)] for i in range(n)]

        return Matrix(inverse)

    def minor(self, i, j):
        """
        获取一个矩阵的余子矩阵，即删除第i行第j列后的矩阵。
        """
        return Matrix([row[:j] + row[j+1:] for row in (self.data[:i] + self.data[i+1:])])

    @property
    def Det(self):
        """
        计算矩阵的行列式。
        """
        if self.rows != self.cols:
            raise ValueError("只有方阵才有行列式")

        # 如果矩阵是1x1，直接返回该元素
        if self.rows == 1:
            return self.data[0][0]

        # 如果矩阵是2x2，使用直接公式计算
        if self.rows == 2:
            return self.data[0][0] * self.data[1][1] - self.data[0][1]*self.data[1][0]

        # 对于更大的矩阵，使用递归计算余子式和代数余子式的和
        det = 0
        for col in range(self.cols):
            # 代数余子式：(-1)(i+j) * a[i][j] * M[i][j]的行列式
            cofactor = ((-1) ** col) * self.data[0][col] * self.minor(0, col).Det
            det += cofactor

        return det
    
    def lu(self):
        if self.rows != self.cols:
            raise ValueError("只有方阵才进行LU分解")
            
        L = [row[:] for row in self.data]
        U = [row[:] for row in self.data]
        A = [row[:] for row in self.data]
        
        n = self.rows
        for i in range(n):
            L[i][i] = 1
            for j in range(i+1, n):
                L[i][j] = 0
                L[j][i] = A[j][i]/A[i][i]
            
            U[i][i] = A[i][i]
            for j in range(i+1, n):
                U[j][i] = 0
                U[i][j] = A[i][j]

            for j in range(i+1, n):
                for k in range(i+1, n):
                    A[j][k] = A[j][k] - L[j][i] * U[i][k]
                    
        return Matrix(L), Matrix(U)
    
    def plu(self):
        if self.rows != self.cols:
            raise ValueError("只有方阵才进行LU分解")
            
        n = self.rows

        L = [row[:] for row in self.data]
        U = [row[:] for row in self.data]
        A = [row[:] for row in self.data]
        p = [i for i in range(0, n)]
        
        for i in range(n):
            
            curline = i
            
            for j in range(i+1, n):
                if abs(A[j][i]) > abs(A[curline][i]): curline = j
                
            if A[curline][i] < 0.00001: raise ValueError("该矩阵无法进行LU分解")
            
            L[i], L[curline] = L[curline], L[i]
            U[i], U[curline] = U[curline], U[i]
            A[i], A[curline] = A[curline], A[i]
            p[i], p[curline] = p[curline], p[i]
            
            
            L[i][i] = 1
            for j in range(i+1, n):
                L[i][j] = 0
                L[j][i] = A[j][i]/A[i][i]
            
            U[i][i] = A[i][i]
            for j in range(i+1, n):
                U[j][i] = 0
                U[i][j] = A[i][j]

            for j in range(i+1, n):
                for k in range(i+1, n):
                    A[j][k] = A[j][k] - L[j][i] * U[i][k]
                    
        return Matrix(L), Matrix(U), Matrix([[1 if i==j else 0 for j in range(0, n)] for i in p])


In [2]:
    data = [[1,2], [2,3]]
    A = Matrix(data)
    print(A)

1 2
2 3


In [4]:
A = Matrix([[1, 2, 3], [4, 5, 6]])
B = Matrix([[7, 8, 9], [10, 11, 12]])
C = Matrix([[13, 14], [15, 16], [17, 18]])

D = Matrix([
[1, 2, 3],
[0, 1, 4],
[5, 6, 0]])

E = Matrix([
[1, 2, 3, 4],
[5, 6, 0, 8],
[9, 10, 11, 12],
[13, 14, 15, 1]])

# 打印矩阵
print(f"A: {A}")
print(f"B: {B}")
print(f"C: {C}")
print(f"D: {D}")
print(f"E: {E}")

A: 1 2 3
4 5 6
B: 7 8 9
10 11 12
C: 13 14
15 16
17 18
D: 1 2 3
0 1 4
5 6 0
E: 1 2 3 4
5 6 0 8
9 10 11 12
13 14 15 1


In [5]:
    A = Matrix([[1, 2, 3], [4, 5, 6]])
    B = Matrix([[7, 8, 9], [10, 11, 12]])
    C = Matrix([[13, 14], [15, 16], [17, 18]])

    D = Matrix([
        [1, 2, 3],
        [0, 1, 4],
        [5, 6, 0]])
    
    E = Matrix([
        [1, 2, 3, 4],
        [5, 6, 0, 8],
        [9, 10, 11, 12],
        [13, 14, 15, 1]])

    # 打印矩阵
    print(f"A: {A}")
    print(f"B: {B}")
    print(f"C: {C}")
    print(f"D: {D}")
    print(f"E: {E}")

    print(f"A + B: { A + B}")
    print(f"A - B: { A - B}")

    print(f"A * 3: {A * 3}")
    print(f"2 * A: {2 * A}")
    print(f"2 * A * 3: {2 * A * 3}")

    print(f"A / 2: {A / 2}")

    # 点积
    print(f"\nA @ C: {A @ C}")

    # 转置
    print(f"A.T: {A.T}")

    print(f'D.Inv: {D.Inv}')

    print(f'D.Inv @ D: {D @ D.Inv}')

    print(f'E.Det: {E.Det}')

A: 1 2 3
4 5 6
B: 7 8 9
10 11 12
C: 13 14
15 16
17 18
D: 1 2 3
0 1 4
5 6 0
E: 1 2 3 4
5 6 0 8
9 10 11 12
13 14 15 1
A + B: 8 10 12
14 16 18
A - B: -6 -6 -6
-6 -6 -6
A * 3: 3 6 9
12 15 18
2 * A: 2 4 6
8 10 12
2 * A * 3: 6 12 18
24 30 36
A / 2: 0.5 1.0 1.5
2.0 2.5 3.0

A @ C: 94 100
229 244
A.T: 1 4
2 5
3 6
D.Inv: -24.0 18.0 5.0
20.0 -15.0 -4.0
-5.0 4.0 1.0
D.Inv @ D: 1.0 0.0 0.0
0.0 1.0 0.0
0.0 0.0 1.0
E.Det: 840


In [11]:
import numpy as np
M = np.arange(1, 10).reshape(3, 3).tolist()
M[0][0] = 0
A = Matrix(M) 
# print(A)
L, U, P = A.plu()

In [12]:
print(L)

1 0 0
0.0 1 0
0.5714285714285714 0.2142857142857144 1
